### Geomorphology Project
Group Members:
1) Sahil Yadav *(23110287)*
2) Pawar Tamanna Sureshbhai *(21110157)*
3) Akhil. P *(24310006)*
4) Chavda Haarit Ravindrakumar *(23110077)*

![Project Intro](./images/terrain_evolution.gif)

## Motivation for selecting the region 

- Beautiful landscape
![Spiti Valley](./images/spiti_img.jpg)
![Spiti Valley](./images/Spiti.png)

**Location:** Himachal Pradesh, Indian Himalayas
**Latitude: 32.0° N to 32.5° N**  
**Longitude: 77.5° E to 78.5° E**  

 

**Why Spiti Valley is good for geomorphic simulation:**

1. **High Elevation (3800 – 4500 m):**  
   The valley is at a high altitude, which helps us clearly see how the land moves upward (uplift) and spreads out (diffusion) over time.

2. **Deep River Valleys and Gorges:**  
   The rivers have cut deep into the land, which makes it easier to model how water flows and collects using tools like flow accumulation.

3. **Glacial Landforms (like snow and ice):**  
   Snow and old glaciers affect how water melts and moves, which is important for studying erosion and river flow.

4. **High Steepness and Slopes:**  
   The steep hills and sharp changes in height help show how fast the land changes due to gravity and water movement.



**Area of the Region**: For change of $1^o$ latitude or longitude it is length of 111km. So for a square of 1 x 1 the area is 111 x 111 ~ $10^{4}km^2$. It is a very large area.


# Landlab Components Overview

## Linear Diffusor
**Use**:  
Models diffusive hillslope processes such as soil creep and sediment transport.

**Governing Equation**:  
The component is based on the linear diffusion equation:
$
\frac{\partial z}{\partial t} = D \nabla^2 z
$
where:  
- $z$ is the topographic elevation,  
- $D$ is the diffusivity constant, and  
- $\nabla^2$ is the Laplacian operator, representing the curvature of the surface.

---

## Flow Accumulator
**Use**:  
Computes the upstream contributing area at each grid cell by routing flow via the steepest descent method (often using algorithms like D8).

**Governing Concept**:  
Instead of a differential equation, it uses flow routing techniques to sum the contributions:
$
A_i = 1 + \sum_{j \in \text{neighbors}} A_j
$
where:  
- $A_i$ is the drainage area at cell $i$, and  
- the sum represents the accumulated area from adjacent upslope cells.

---

## Stream Power Eroder
**Use**:  
Simulates fluvial incision and channel erosion based on stream power, capturing how water discharge and slope drive erosion.

**Governing Equation**:  
It is governed by the stream power law:
$
\frac{\partial z}{\partial t} = -K A^m S^n
$
where:  
- $K$ is an erodibility coefficient,  
- $A$ is the drainage area,  
- $S$ is the local channel slope, and  
- $m$ and $n$ are empirically-determined exponents.

---

## Steepness Finder
**Use**:  
Calculates channel steepness indexes for assessing landscape evolution and tectonic activity, often using the derived slopes and drainage areas.

**Governing Concept**:  
Typically, the relationship between slope and area is expressed as:
$
S = k_s A^{-\theta}
$
where:  
- $S$ is the channel slope,  
- $A$ is the drainage area,  
- $k_s$ is the channel steepness index, and  
- $\theta$ is the concavity index.

---



---

### Conservation Laws

**1. Conservation of Mass**  
- **Principle:** Mass is neither created nor destroyed within a closed system.  
**2. Conservation of Linear Momentum**  
- **Principle:** The linear momentum of a closed system remains constant unless acted upon by external forces.  
---

### Constitutive Relationships

**1. Linear Diffusion**  
- **Description:** This relationship describes the process whereby particles or properties such as heat or solute concentration spread out over time.  

**2. Fluvial Erosion**  
- **Description:** This relationship quantifies how flowing water removes material from the land surface.  

**3. Channel Steepness**  
- **Description:** Channel steepness reflects the slope or gradient of a river channel, which influences the sediment transport capacity and erosion potential.  

**4. Glacial Erosion**  
- **Description:** Glacial erosion involves the mechanical removal of rock and sediment by the movement of glaciers.  



---

**Landlab: Principle, Theory, and Uses**  
- **Principle**: Grid-based framework using numerical methods to solve PDEs. Multiple components (e.g., erosion, diffusion) interact on shared grids.  
- **Theory**: Based on conservation laws, constitutive relationships, and numerical methods.  
- **Uses**: Simulates landscape evolution, river incision, hillslope diffusion, and tectonic processes.

---

**Challenges in Application**  
- Complex boundary condition setup  
- High sensitivity to input parameters  
- Runtime and memory limitations  
- DEMs often lack fine-scale detail  
- Array handling issues (e.g., unraveling problems)

---

**Key Learnings**  
- **Process-Based Modeling**: Landscapes are dynamic systems shaped by physical laws.  
- **Hillslope & Fluvial Processes**: Learned how diffusion smoothens terrain and erosion forms valleys.  
- **Modular Numerical Modeling**: Gained experience using plug-and-play components like `FlowAccumulator`, `LinearDiffuser`, and `StreamPowerEroder`.  
- **Topography as Data**: Treated DEMs as evolving datasets, not just static maps.  
- **Importance of Boundary Conditions**: Small changes in parameters or initial topography can drastically alter outcomes, reflecting real-world complexity.

---


# Code Walk Through

In [ ]:
import bmi_topography
from bmi_topography import Topography
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, FastscapeEroder, LinearDiffuser, SinkFiller, ChiFinder, SteepnessFinder
from landlab.plot import imshow_grid
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import imageio
import os
from itertools import product

def getTerrain(south, north, west, east, cache_dir="."):
    """Download and load DEM data using bmi-topography."""
    topo = Topography(
        dem_type="SRTMGL3",
        south=south, north=north,
        west=west, east=east,
        output_format="GTiff",
        cache_dir=cache_dir,
    )
    topo.fetch()
    da = topo.load()
    da.rio.write_crs("EPSG:4326", inplace=True)
    return da

# Define DEM spatial limits (Baspa River area, Himachal Pradesh, India)
lat_center, lon_center = 32, 78
buffer = 0.5  # degrees
south = lat_center - buffer
north = lat_center + buffer
west = lon_center - buffer
east = lon_center + buffer

# Simulation parameters
dt = 500                 # Time step in years
nr_time_steps = 1000     # Number of simulation steps
xy_spacing = 90          # Grid cell spacing (meters)

# Parameter arrays for simulation runs
K_sp_vals       = [2e-8, 5e-6]    # Stream power erodibility
m_sp_vals       = [0.5, 0.7]       # Area exponents
n_sp_vals       = [1.0, 1.2]       # Slope exponents
diffusivity_vals = [0.3, 0.5]      # Diffusivity (m^2/yr)
uplift_vals     = [0.1, 0.001]     # Uplift rate (m/yr)

# Download and prepare the DEM
da = getTerrain(south, north, west, east)
dem_raw = da.data[0]
# Replace any NaN values with the minimum valid elevation
dem_clean = np.where(np.isnan(dem_raw), np.nanmin(dem_raw), dem_raw)
dem_master = np.array(dem_clean, dtype=np.float64)
nrows, ncols = dem_master.shape

# Loop over all parameter combinations
for K_sp_val, m_sp_val, n_sp_val, diffusivity_val, uplift_val in product(
        K_sp_vals, m_sp_vals, n_sp_vals, diffusivity_vals, uplift_vals):

    # Create an identifier for the parameter combination
    param_tag = f"ksp_{K_sp_val:.0e}_msp_{m_sp_val}_nsp_{n_sp_val}_diff_{diffusivity_val}_uplift_{uplift_val:.0e}"
    print(f"\n--- Starting simulation: {param_tag} ---")
    
    # Create a directory for output frames
    output_dir = f"frames_{param_tag}"
    os.makedirs(output_dir, exist_ok=True)

    # Set up simulation grid and initial topography
    dem = np.copy(dem_master)
    mg = RasterModelGrid((nrows, ncols), xy_spacing=xy_spacing)
    z = mg.add_field('topographic__elevation', dem.flatten(), at='node')

    # Initialize Landlab components
    sf = SinkFiller(mg, apply_slope=True)
    fr = FlowAccumulator(mg, flow_director='D8')
    ld = LinearDiffuser(mg, linear_diffusivity=diffusivity_val)
    fse = FastscapeEroder(mg, K_sp=K_sp_val, m_sp=m_sp_val, n_sp=n_sp_val)
    chif = ChiFinder(mg, min_drainage_area=0.0, reference_concavity=m_sp_val/n_sp_val)
    steepnessf = SteepnessFinder(mg, reference_concavity=m_sp_val/n_sp_val)

    # Set uplift parameters
    rock_up_rate = uplift_val
    rock_up_len = rock_up_rate * dt
    total_time = 0
    frame_count = 0

    # Define plot limits for consistent color mapping
    initial_min_z = np.min(z)
    initial_max_z = np.max(z)
    max_possible_z = initial_max_z + rock_up_rate * dt * nr_time_steps
    plot_vmin = initial_min_z
    plot_vmax = max_possible_z

    # Run the main simulation loop
    for i in range(nr_time_steps):
        z[mg.core_nodes] += rock_up_len   # Apply uplift
        ld.run_one_step(dt)               # Diffuse hillslopes
        sf.run_one_step()                 # Fill depressions
        fr.run_one_step()                 # Route flow
        fse.run_one_step(dt)              # Erode channels
        
        total_time += dt
        
        # Print progress intermittently
        if i % (nr_time_steps // 20) == 0 or i == nr_time_steps - 1:
            print(f"[{param_tag}] Step {i}/{nr_time_steps}, Time: {total_time / 1000:.1f} kyr")
        
        # Save simulation frames intermittently (~50 frames)
        if i % (nr_time_steps // 50) == 0 or i == nr_time_steps - 1:
            plt.figure(figsize=(10, 8))
            imshow_grid(mg, 'topographic__elevation',
                        plot_name=f'Elevation at {total_time / 1000:.1f} kyr',
                        allow_colorbar=True, cmap='terrain',
                        vmin=plot_vmin, vmax=plot_vmax)
            frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.png")
            plt.savefig(frame_filename)
            plt.close()
            frame_count += 1

    # Create an animated GIF of the simulation run
    gif_filename = f"terrain_evolution_{param_tag}.gif"
    try:
        with imageio.get_writer(gif_filename, mode='I', duration=0.2) as writer:
            for i in range(frame_count):
                frame_path = os.path.join(output_dir, f"frame_{i:04d}.png")
                if os.path.exists(frame_path):
                    image = imageio.v2.imread(frame_path)
                    writer.append_data(image)
        print(f"[{param_tag}] GIF saved as {gif_filename}")
    except Exception as e:
        print(f"[{param_tag}] Error creating GIF: {e}")

    # Final flow routing and channel analysis
    sf.run_one_step()
    fr.run_one_step()
    try:
        steepnessf.calculate_steepnesses(min_drainage_area=xy_spacing**2)
        chif.calculate_chi(min_drainage_area=xy_spacing**2)
    except Exception as e:
        print(f"[{param_tag}] Error in analysis: {e}")

    # Plot final topography results
    def plot_grid_final(field_name, title, label=None, cmap='viridis', vmin=None, vmax=None, use_log=False):
        plt.figure(figsize=(10, 8))
        data = mg.at_node[field_name]
        if use_log:
            valid_data = data[data > 0]
            if len(valid_data) == 0:
                plot_data = np.zeros_like(data)
            else:
                plot_data = np.log10(np.maximum(data, np.min(valid_data)*0.1))
                if label:
                    label = f"Log10({label})"
        else:
            plot_data = data

        if vmin is None and vmax is None:
            valid_plot_data = plot_data[np.isfinite(plot_data)]
            if len(valid_plot_data) > 0:
                vmin = np.percentile(valid_plot_data, 2)
                vmax = np.percentile(valid_plot_data, 98)
            else:
                vmin, vmax = 0, 1

        imshow_grid(mg, plot_data, plot_name=title,
                    allow_colorbar=True, colorbar_label=label,
                    cmap=cmap, vmin=vmin, vmax=vmax)
        plt.show()

    plot_grid_final(
        'topographic__elevation',
        f'Final Topography ({total_time / 1e6:.2f} Myr) - {param_tag}',
        label='Elevation (m)', cmap='terrain',
        vmin=plot_vmin, vmax=plot_vmax
    )
    # Optionally remove the frame files after GIF creation
    # shutil.rmtree(output_dir)

print("All simulations finished.")
